In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline
import os

import os

from utils_behavior import ProcessSession
import binarize2pcalcium

Autosaving every 180 seconds


In [2]:

def run_binarization_ensemble(root_dir, animal_id, session_id,
                              clip_windows = None,
                              dff_min = 5,
                              use_suite2p=False,
                              maximum_std_of_signal=0.08):
    ########################################################
    # load the BMI enesemble data
    ########################################################
    sess = ProcessSession(root_dir,
                        animal_id,
                        session_id)

    #
    sess.session_id = session_id
    sess.session_duration = 30*60*30
    sess.skip_dict = True
    sess.verbose=True

    #
    sess.rec_len_mins = 30
    sess.load_data()

    # convert the ensembls into a stack
    sess.E1[0][:5 * sess.sample_rate] = 0
    sess.E1[1][:5 * sess.sample_rate] = 0
    sess.E2[0][:5 * sess.sample_rate] = 0
    sess.E2[1][:5 * sess.sample_rate] = 0

            #
    traces = np.vstack((sess.E1[0],
                        sess.E1[1],
                        sess.E2[0],
                        sess.E2[1])).astype('float32')
    #
    sess.F = traces.copy()

    # replace the BMI real time signal with the suite2p time corcted one
    if use_suite2p:

        from calcium import Calcium

        c = Calcium()
        c.data_dir = os.path.join(sess.root_dir,
                                  sess.animal_id,
                                  sess.session_id,
                                  'plane0')
        c.load_suite2p()

        # find each of the matching traces
        for k in range(sess.F.shape[0]):
            temp1 = sess.F[k]
            temp1 = (temp1 - np.mean(temp1))/np.mean(temp1)

            pcorr_max = 0
            pcorr_id = None
            match_trace = None
            for p in range(c.F.shape[0]):
                temp3 = c.F[p,:sess.session_duration]
                # remove mean
                temp3 = (temp3 - np.mean(temp3))/np.mean(temp3)
                # compute the correlation
                temp2 = np.corrcoef(temp1, temp3)
                if temp2[0, 1] > pcorr_max:
                    pcorr_max = temp2[0, 1]
                    #pcorr_id = p
                    match_trace = temp3
            #print ("found match: ", pcorr_id, pcorr_max)
            
            # #
            # plt.figure()
            # plt.plot(temp1)
            # plt.plot(match_trace)

            sess.F[k] = match_trace.copy()


    ###################################################
    #
    import binarize2pcalcium.binarize2pcalcium as binca

    #
    c = binca.Binarize(root_dir, animal_id, session_id)

    c.session = session_id 
    c.session_name = session_id
    c.rec_len_mins = 30
    c.bin_width = 5

    c.data_type = '2p'
    c.remove_bad_cells = False
    c.verbose = False                          # outputs additional information during processing
    c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations

    # set flags to save matlab and python data
    c.save_python = True         # save output as .npz file 
    c.save_matlab = False         # save output as .mat file

    # manual thresholds for spike detection
    c.dff_min = dff_min                 # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
    c.percentile_threshold = 1-1E-8   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
    c.maximum_std_of_signal = maximum_std_of_signal     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it
                                        #    - this is a very important flag! come see me if you don't understand it

    #
    c.F = sess.F.copy()
    c.data_dir = os.path.join(root_dir, animal_id, session_id)
    c.yaml_file_exists=False
    c.recompute_binarization = True
    c.fname_out = os.path.join(c.data_dir, 
                               'results',
                               'binarized.npz')

    #
    c.moment_flag = False
    c.mode_window = None #30*1240
    #c.remove_ends = False
    #c.remove_starts = True
    #c.remove_length = clip_window*30
    c.clip_windows = clip_windows
    c.min_width_event_upphase = c.sample_rate//3
    c.binarize_data()


In [3]:
####################################################
################## LOAD DATA #######################
####################################################

animal_ids = [
    # #M1 mice
    #"DON-011733",    # M1    - Processed
    #  'DON-014382',    # M1    - Processed
    #   'DON-014451',    # M1    - Processed
    #   'DON-014575',    # M1    - Processed
    #  "DON-014618",    # M1    - Processed
      #'DON-015821',    # M1    - Processed
     #  'DON-017923',   # M1    - Not Processed <has extra days at the end....
    #   'DON-018129',   # M1    - Not Processed
    # 'DON-018130',   # M1    - Not Processed

    # #CA3 mice
     # "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
     # "DON-014266",    # CA3   - Processed
      #'DON-014371',    # CA3   - Processed
    # 'DON-015801',    # CA3   - Processed
      'DON-016658',    # CA3   - Processed
    # 'DON-019683',    # CA3   - NOT Processed

]

# root_dir = r'F:\bmi\cohort8'
root_dir = '/media/cat/8TB/donato/bmi/'
animal_id = animal_ids[0]

# load yaml file for the animal
import yaml
with open(os.path.join(root_dir,
                       animal_id,
                       animal_id+'.yaml')) as file:
    doc = yaml.load(file, Loader=yaml.FullLoader)

try:
    session_ids = np.array(doc['session_names'],dtype='str')
except:
    session_ids = np.array(doc['session_ids'],dtype='str')

#
session_id = session_ids[8]

#

clip_windows = [[0,10],
                #[1050,1100]
                ]

dff_min = 7.5
use_suite2p = True
maximum_std_of_signal = 0.08


#
run_binarization_ensemble(root_dir, 
                          animal_id, 
                          session_id,
                          clip_windows,
                          dff_min,
                          use_suite2p,
                        maximum_std_of_signal)


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


remove bad cells:  True
...setting 2p parameters...
... computing binarization on non yaml-based data...


binarizing continuous traces : 100%|██████████| 4/4 [00:00<00:00, 303.83it/s]


...saving data...
...saving figures...


In [128]:
# ########################################################
# ############# BINARIZE THE [CA] TRACES #################
# ########################################################

# # double check that we have set the STD thrshold at a reasonable level to catch biggest/highest SNR bursts
# sess.show_plots = True

# #
# sess.visualize_binarization()

In [1]:
# ############################################################
# ############# PLOTS IN HONOR OF CAJAL AND RY ###############
# ############################################################
# sess.show_plots=True
sess.process_session_traces()

NameError: name 'sess' is not defined

In [14]:
############################################################
############# PLOTS IN HONOR OF CAJAL AND RY ###############
############################################################

#
#sess.load_calibration()

# run this if you have run the calibration through the bmi gui
#sess.load_calibration_new()

#
sess.show_plots=True
sess.show_session_traces_and_calibration()

#F:\bmi\cohort8\DON-014451\20230328\calibration
#F:\bmi\cohort8\DON-014451\20230328\\results.npz'

AttributeError: 'ProcessSession' object has no attribute 'e1_1'

In [5]:
#################################################################
############# CORREALTION BETWEEN ENSEMBLES #####################
#################################################################
# this is not so interesting; may need to do more on this
sess.show_plots = True

#
sess.compute_ensemble_correlations()


pearson correlation E1 cells;  0.03916222897242536
pearson correlation E2 cells;  0.09556712722981311


In [6]:
#################################################################
############# CORRELOGRAMS BETWEEN REWARDS AND LICKING ##########
#################################################################
# this is not so interesting; may need to do more on this
sess.show_plots = True
#
sess.window_size = 60   # seconds
sess.compute_correlograms_reward_vs_licking()


lick times:  [   3.18    3.19    3.2  ... 2985.97 2985.98 2985.99]
# of spikes:  12406


model filter: remove bleaching or trends: 100%|██████████| 4/4 [00:00<00:00, 1789.19it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 4/4 [00:00<00:00, 434.89it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 4/4 [00:00<00:00, 172.19it/s]


In [20]:
###################################################################################
############COMPUTE CORRELOGRAMS BETWEEN CELLS BASED ON UPPHASE DETECTION #########
###################################################################################
#
sess.corr_window = 60   # in seconds
sess.compute_correlograms_ensembles_upphase()

In [2]:
###########################################################################################
############COMPUTE CORRELOGRAMS BETWEEN CELLS BASED ON RAW FILTERED FLUORESCENCE #########
##########################################################################################
#
sess.show_plots = True
sess.window = 60*30   # in seconds
sess.compute_correlograms_ensembles_fluorescence()

NameError: name 'sess' is not defined

In [11]:
########################################################
####### COMPUTE INTRA-SESSION REWARD HISTOGRAM #########
########################################################

#
sess.show_plots=True
sess.bin_width = 5 # in minutes
sess.compute_intra_session_reward_histogram()



Perason corr:  PearsonRResult(statistic=0.3706110830816283, pvalue=0.2917649089674079)


In [12]:
######################################################################
####### COMPUTE INTRA-SESSION REWARD HISTOGRAM - HITS/MINUTE #########
######################################################################

#
sess.show_plots=True
sess.bin_width_mins = 5 # in minutes
sess.compute_intra_session_reward_histogram_hist_per_minute()

Perason corr:  PearsonRResult(statistic=0.37154698242568246, pvalue=0.2904522471458872)


In [13]:
##########################################################################
####### COMPUTE INTRA-SESSION CELL BURST HISTOGRAMS AS LINE PLOT #########
##########################################################################

#
sess.show_plots=True
sess.bin_width = 5 # in minutes
sess.compute_intra_session_cell_burst_histogram()


In [14]:
##########################################################################
####### COMPUTE INTRA-SESSION CELL BURST HISTOGRAMS AS LINE PLOT #########
##########################################################################
#
sess.compute_intra_session_cell_burst_histogram_v2()

In [7]:
####################################################
####### INTER-BURST-INTERVAL DISTRIBUTIONS #########
####################################################
#
sess.isi_width = 10
sess.isi_bin_width = 0.250
sess.compute_intra_session_inter_burst_interval()

In [67]:
#######################################
####### TIME TO REWARD PER TRIAL ######
#######################################

#
sess.time_to_reward_per_trial_intra_session()



nan


In [161]:
###########################################
####### BINNED PAIRWISE CORRELATIONS ######
###########################################

#
sess.session_id = '20230202'
#sess.session_id = '20230131'
sess.pairwise_correlations_intra_session_dff()



In [207]:
##################################
####### LICKS VS. ENSEMBLES ######
##################################

#
sess.session_id = '20230202'
#sess.session_id = '20230131'
sess.load_data()

# not super informative
#sess.compute_licks_vs_ensembles()


#
sess.window =30*30  # 30 seconds
sess.compute_licks_vs_ensembles_correlograms()


missing:  abs_times_ttl_read
missing:  abs_times_ca_read
matching lick times to ttl Times:  (89955,)


100%|██████████| 89955/89955 [00:35<00:00, 2537.89it/s] 


reward times:  (31,)
Recording length (mins):  50.0
abs times:  (3008338,) [ 149.8581727  149.8583522  149.8585126 ... 3156.1436586 3156.1453174
 3156.1455295]
ttl times:  (89955,) 0.0 2999.9520262  total rec time sec:  2999.9520262
ttl computed:  (89955,) [    0     1     2 ... 89997 89998 89999]
ttl detected:  (89955,) [    0     1     2 ... 89952 89953 89954]
lick detector:  (3008338, 1)
lick times:  [ 587.9077456  587.9079715  587.9097345 ... 2703.8404269 2703.8421129
 2703.8424087]
E1 , E2,  (2, 90000) (2, 90000)
(89956,) (89956,)


100%|██████████| 1800/1800 [00:04<00:00, 416.59it/s]


In [199]:
d = np.load('/media/cat/4TB/donato/andres/DON-011733/20230202/data/lick_times_ttl.npy')
print (d.shape)

(89956,)


In [160]:
# ###################################################
# ######## LOAD ROI CONTOURS AND VISUALIZE ##########
# ###################################################

# data = np.load('/media/cat/4TB/donato/andres/DON-011733/20230127/data/results.npz')
# rois_e1 = data['rois_pixels_ensemble1'].T
# rois_e2 = data['rois_pixels_ensemble2'].T
# print (rois_e1.shape)
# plt.figure()

# plt.scatter(rois_e1[:,0], rois_e1[:,1])
# plt.scatter(rois_e2[:,0], rois_e2[:,1])
# plt.xlim(0,512)
# plt.ylim(0,512)
# plt.show()
# # #

(490, 2)


In [22]:
####################################################
################ SESSION % HITS ####################
####################################################

root_dir = '/media/cat/4TB/donato/andres/Cohort 6 Data/'
animal_id = 'DON-014266'
session_ids = [
    #'20230127',
    #'20230128',
    #'20230129',
    #'20230130',
    '20230131',
    '20230201',
    '20230202',
    '20230203',
    '20230204',
    '20230205',
    '20230206',
    #'20230207',

]

# define the 3 sets of sessions for contingency degradation
# leave blank for non-used
t = ['20230131',
        '20230201']
cd = ['20230202',
        '20230203']
r = ['20230205',
        '20230206']


# define early vs. late sessions
early = ['20230131',
        '20230201',
        '20230202',
        ]
late = ['20230204',
        '20230205',
        '20230206',
       ]

In [23]:
#####################################################
#####################################################
#####################################################
      
sess = ProcessSession(root_dir,
                      animal_id,
                     session_ids[0])

sess.load_data()

#
sess.bin_width_mins = 5
sess.verbose=False
sess.session_ids = session_ids

sess.compute_animal_hit_rate()

In [17]:
import numpy as np
fname = '/media/cat/8TB/donato/bmi/DON-014618/20230505/results.npz'
data = np.load(fname, allow_pickle=True)    

abs_times = data['abs_times_ttl_read']
ttl_times = data['ttl_times']

lick_detector = data['lick_detector_abstime']
idx = np.where(lick_detector > 3)[0]
lick_times = abs_times[idx] - ttl_times[0]
print (lick_times)

dd = data['abs_times_ca_read']
print (dd.shape)


[2.43144700e-01 2.43368100e-01 2.43546400e-01 ... 2.86109859e+03
 2.86109895e+03 2.86110055e+03]
(90000,)


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml
%matplotlib tk


animal_ids = [
    # #M1 mice
    "DON-011733",    # M1    - Processed
     'DON-014382',    # M1    - Processed
     'DON-014451',    # M1    - Processed
    'DON-014575',    # M1    - Processed
    "DON-014618",    # M1    - Processed
     'DON-015821',    # M1    - Processed
    'DON-017923',   # M1    - Not Processed <has extra days at the end....
    'DON-018129',   # M1    - Not Processed
    'DON-018130',   # M1    - Not Processed

    # #CA3 mice
    "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
    "DON-014266",    # CA3   - Processed
    'DON-014371',    # CA3   - Processed
    'DON-015801',    # CA3   - Processed
    'DON-016658',    # CA3   - Processed
    'DON-019683'
]

root_dir = '/media/cat/8TB/donato/bmi/'

for animal_id in animal_ids:
    # load yaml file from this location
    fname = os.path.join(root_dir, animal_id, 
                        str(animal_id)+'.yaml')
    yam = yaml.load(open(fname, 'r'), Loader=yaml.FullLoader)
    # grab "session_ids" from yaml file
    sessions = yam['session_ids']



    # make color map over 7 discrete viridis
    cmap = plt.cm.viridis
    colors = cmap(np.linspace(0, 1, len(sessions)-1))
    cell_names = ['pos1', 'pos2', 'neg1', 'neg2']
    #
    plt.figure(figsize=(8,8))
    cell_array = [[],[],[],[]]
    for ctr,session in enumerate(sessions[1:]):
        fname = os.path.join(root_dir,
                            animal_id, 
                            str(session), 
                            'results',
                            'cell_burst_histogram_v2.npy')
        data = np.load(fname, allow_pickle=True)
        print (data.shape)
        t = np.arange(6)*5+2.5
        for k in range(data.shape[0]):
            ax=plt.subplot(2,2,k+1)
            plt.plot(t,
                    data[k],
                    color=colors[ctr],
                    linewidth=2,
                    )
            
            plt.title("Cell: "+str(k))
            cell_array[k].append(data[k])

    # plot also average of all cells
    for k in range(4):
        ax=plt.subplot(2,2,k+1)
        temp = np.array(cell_array[k])
        std = temp.std(0)
        temp = temp.mean(0)
        plt.ylabel("burst rate(5min bins)")
        plt.plot(t,
                temp, 
                color='black',
                linewidth=2)
        
        #
        plt.fill_between(t,
                        temp-std,
                        temp+std,
                        color='black',
                        alpha=0.2)
        plt.title("Cell: "+str(cell_names[k]))

    plt.suptitle("Cell burst histograms " + animal_id)
    plt.show()

    # 
    plt.savefig('/home/cat/'+animal_id+'_cell_burst_histograms.png')

(4, 50)


ValueError: x and y must have same first dimension, but have shapes (6,) and (50,)